In [1]:
###### Importo librerias y leo el archivo limpio luego de terminar el ETL  ######
import pandas as pd
df = pd.read_csv("DataSets/peliculas.csv",low_memory=False)

In [42]:
###### Visualiso los datos y empieso a analizarlos y ver que columasn me sirven y cuales no ######
df.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,return,month_name,day_name
0,Toy Story Collection,30000000,"['Animation', 'Comedy', 'Family']",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,['Pixar Animation Studios'],['United States of America'],1995-10-30,373554033.0,81.0,['English'],Released,NaN,Toy Story,7.7,12.0,Octubre,Lunes
1,NaN,65000000,"['Adventure', 'Fantasy', 'Family']",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"['TriStar Pictures', 'Teitler Film', 'Intersco...",['United States of America'],1995-12-15,262797249.0,104.0,"['English', 'Français']",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,4.0,Diciembre,Viernes


In [48]:
###### Luego del analisis inicial elimino las columnas que veo son innecesarias para el modelo de Machine Learning #######
###### id se elimina por que es solo un identificador los registros dentro de una base de datos ######
df.drop("id",axis="columns",inplace=True)

###### overview y tagline son columnas con descripciones en texto muy largas y dificiles de procesar para mi modelo de ML ######
df.drop("overview",axis="columns",inplace=True)
df.drop("tagline",axis="columns",inplace=True)

###### month_name y day_name las elimino por que solo las cree para una consulta especifica no para el modelo de ML ######
df.drop("month_name",axis="columns",inplace=True)
df.drop("day_name",axis="columns",inplace=True)

###### Al analizar la fecha me doy cuenta que no es necesario el dia, ni el mes y en cuanto al año lo normalizo por decada ######
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_date'] = (df['release_date'].dt.year // 10) * 10

###### Observo que las columnas popularity,vote_average_runtime son numericas y tienen muchas variaciones ######
###### genero quartiles para poder categorizarlas y les doy una clasificaion para poder procesarlas mas facilmente en mi modelo ######
df['popularity'] = pd.qcut(df['popularity'], q=7, labels=["Muy Malo",'Malo',"Normal",'Bueno','Muy bueno',"Exelente","Obra Maestra"])
df['vote_average'] = pd.qcut(df['vote_average'], q=7, labels=["Muy Malo",'Malo',"Normal",'Bueno','Muy bueno',"Exelente","Obra Maestra"])
df['runtime'] = pd.qcut(df['runtime'], q=6, labels=["Muy poco",'Poco',"Normal",'Mucho','Demasiado',"Extremadamente largo"])
df['revenue'] = pd.qcut(df['revenue'].rank(method='first'), q=6, labels=["Muy poco",'Poco',"Normal",'Mucho','Demasiado',"Extremo"])

###### Uso la funcion de pandas "Categorical" para reemplazar las variables en texto a su respectivo indice que genera la funcion ######
df["popularity"] = pd.Categorical(df["popularity"]).codes
df["vote_average"] = pd.Categorical(df["vote_average"]).codes
df["runtime"] = pd.Categorical(df["runtime"]).codes
df["revenue"] = pd.Categorical(df["revenue"]).codes

df["belongs_to_collection"] = pd.Categorical(df["belongs_to_collection"]).codes
df["genres"] = pd.Categorical(df["genres"]).codes
df["original_language"] = pd.Categorical(df["original_language"]).codes
df["production_companies"] = pd.Categorical(df["production_companies"]).codes
df["production_countries"] = pd.Categorical(df["production_countries"]).codes
df["spoken_languages"] = pd.Categorical(df["spoken_languages"]).codes
df["status"] = pd.Categorical(df["status"]).codes

In [44]:
df.head(2)

,belongs_to_collection,budget,genres,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,return
0,1546,30000000,1090,17,6,15264,2370,1990,373554033.0,1,906,4,Toy Story,6,12.0
1,-1,65000000,941,17,6,19691,2370,1990,262797249.0,4,626,4,Jumanji,5,4.0


In [49]:
###### Visualiso el diagrama de correlaciones del dataframe ######
corr = df.corr()
corr.style.background_gradient(cmap="coolwarm")

C:\Users\Matias\AppData\Local\Temp\ipykernel_9568\1928674225.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


,belongs_to_collection,budget,genres,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,vote_average,return
belongs_to_collection,1.000000,0.162048,-0.054747,0.017188,0.170220,0.077845,0.057763,0.012526,0.067462,0.000970,0.060628,0.004817,0.014416,-0.001121
budget,0.162048,1.000000,-0.090666,-0.071830,0.323036,0.139549,0.111895,0.130426,0.286263,0.195758,-0.010075,0.007682,0.055189,-0.002137
genres,-0.054747,-0.090666,1.000000,-0.008196,0.033544,0.064655,0.094196,-0.004030,-0.029909,0.051318,0.051467,0.020508,-0.006413,0.002901
original_language,0.017188,-0.071830,-0.008196,1.000000,-0.090769,-0.012819,-0.098816,0.047975,0.024318,0.088016,0.361628,0.003955,0.096792,-0.001384
popularity,0.170220,0.323036,0.033544,-0.090769,1.000000,0.286174,0.287538,0.197407,0.111752,0.304495,0.100043,0.028883,0.220921,0.008622
production_companies,0.077845,0.139549,0.064655,-0.012819,0.286174,1.000000,0.347081,-0.169780,0.036250,0.123400,0.118042,0.033309,0.081665,0.007603
production_countries,0.057763,0.111895,0.094196,-0.098816,0.287538,0.347081,1.000000,-0.188646,0.010763,0.059173,0.142983,0.032019,0.020538,0.006551
release_date,0.012526,0.130426,-0.004030,0.047975,0.197407,-0.169780,-0.188646,1.000000,0.244873,0.104617,0.033714,-0.030638,-0.020491,-0.005471
revenue,0.067462,0.286263,-0.029909,0.024318,0.111752,0.036250,0.010763,0.244873,1.000000,0.025071,0.017479,-0.013465,-0.015547,0.012936
runtime,0.000970,0.195758,0.051318,0.088016,0.304495,0.123400,0.059173,0.104617,0.025071,1.000000,0.160598,0.022259,0.190581,0.000217


In [50]:
###### En el diagrama observe que hay una correlacion moderada en distintas columnas por ende las elimino para no tener redundancia de datos ######
###### Budget y revenue ya estan implicitas en return y tienen corelacion con polularity elimino las dos ######
df.drop("budget",axis="columns",inplace=True)
df.drop("revenue",axis="columns",inplace=True)

###### production_countries correlacion con production_companies elimino la primera ######
df.drop("production_countries",axis="columns",inplace=True)

###### spoken_languages correlacion con original_language elimino la primera ######
df.drop("spoken_languages",axis="columns",inplace=True)

In [51]:
###### Visualiso el diagrama de correlaciones del dataframe ######
corr = df.corr()
corr.style.background_gradient(cmap="coolwarm")

C:\Users\Matias\AppData\Local\Temp\ipykernel_9568\1928674225.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


,belongs_to_collection,genres,original_language,popularity,production_companies,release_date,runtime,status,vote_average,return
belongs_to_collection,1.000000,-0.054747,0.017188,0.170220,0.077845,0.012526,0.000970,0.004817,0.014416,-0.001121
genres,-0.054747,1.000000,-0.008196,0.033544,0.064655,-0.004030,0.051318,0.020508,-0.006413,0.002901
original_language,0.017188,-0.008196,1.000000,-0.090769,-0.012819,0.047975,0.088016,0.003955,0.096792,-0.001384
popularity,0.170220,0.033544,-0.090769,1.000000,0.286174,0.197407,0.304495,0.028883,0.220921,0.008622
production_companies,0.077845,0.064655,-0.012819,0.286174,1.000000,-0.169780,0.123400,0.033309,0.081665,0.007603
release_date,0.012526,-0.004030,0.047975,0.197407,-0.169780,1.000000,0.104617,-0.030638,-0.020491,-0.005471
runtime,0.000970,0.051318,0.088016,0.304495,0.123400,0.104617,1.000000,0.022259,0.190581,0.000217
status,0.004817,0.020508,0.003955,0.028883,0.033309,-0.030638,0.022259,1.000000,0.019083,0.000290
vote_average,0.014416,-0.006413,0.096792,0.220921,0.081665,-0.020491,0.190581,0.019083,1.000000,0.005641
return,-0.001121,0.002901,-0.001384,0.008622,0.007603,-0.005471,0.000217,0.000290,0.005641,1.000000


In [52]:
###### Finalmente terminado el EDA genero un nuevo csv limpio para el consumo del modelo de Machine Learning ######
df.to_csv("DataSets/PeliculasML.csv",index=False)